## Solving Systems of Linear Equations


Recall the three basic rules for matrix manipulation from linear algebra:

 1. Switching two rows or columns does not change the solution of the linear system.
 1. Any row can be multiplied by a constant without changing the solution of the linear system.
 1. Any row or linear multiple of a row can be added/subtracted to/from another row without changing the solution of the linear system.
        
Consider the system of equations:

$$3 x_1 + 8 x_2 = 10$$
   
$$4 x_1 + 6 x_2 = 2$$

We can write in matrix form:

$$
\begin{bmatrix}
3 & 8 & 10\\ 4 & 6 & 2
\end{bmatrix}
$$

We can multiply the 1st row $\left(R_1\right)$ by $-\frac{4}{3}$ and add it to the 2nd row $\left(R_2\right)$, etc...

$$
\begin{bmatrix}
3 & 8 & 10 \\ 4 & 6 & 2
\end{bmatrix} \xrightarrow{-\frac{4}{3}R_1 \to R_1}
\begin{bmatrix}
-4 & -\frac{32}{3} & -\frac{40}{3} \\ 4 & 6 & 2
\end{bmatrix} \xrightarrow{R_1 + R_2 \to R_2}
\begin{bmatrix}
-4 & -\frac{32}{3} & -\frac{40}{3} \\ 0 & -\frac{14}{3} & -\frac{34}{3}
\end{bmatrix} {\xrightarrow[-\frac{3}{14}R_2 \to R_2]{\frac{1}{4}R_1 \to R_1}}
\begin{bmatrix}
1 & \frac{8}{3} & \frac{10}{3} \\ 0 & 1 & \frac{17}{7}
\end{bmatrix}
$$

By inspection we can see that $x_2 = \frac{17}{7}$ we can then substitue this solution into a modified first equation

$$
x_1 + \frac{8}{3}x_2 = \frac{10}{3}
$$

and solve for $x_1$. The final solution is $x_1 = -\frac{22}{7}$,  $x_2 = \frac{17}{7}$. 

This procedure is called Gaussian elimination.

## Stability of Gaussian Elimination

Recall that another way of writing a linear system in matrix form in $A\overline{x} = \overline{b}$ where $A$ is the coefficient matrix, $\overline{x}$ is the vector of unknowns, and $\overline{b}$ is the right-hand side vector.

One method of determining if a linear system has a solution is to take the determinate of the $A$ matrix. If the determinate of $A = 0$ then the matrix is said to be *singular* and the system either has no solution or an infinite number of solutions.

A matrix that is near-singular is said to be *ill-conditioned* whereas one that is far from singular is said to be *well-conditioned*. There is a notion of *condition number* that mathematically quantifies a matrix's amenability to numeric computations with matrices having high condition numbers being well condidioned and matrices with low condition numbers being ill conditioned. We will return to condition numbers in more detail when we discuss singlar value decomposition. Below is an example of two matrices $A$ and $B$. $A$ is an ill-conditioned matrix, $B$ is well-conditioned.

In [1]:
import numpy as np
A = np.array([[1, 1], [1, 1.001]])
print(A)

np.linalg.det(A)

[[1.    1.   ]
 [1.    1.001]]


0.00099999999999989

If we change the last entry of $A$ to $a_{22} = 1$, it is singular.

In [12]:
A = np.matrix('1 1;1 1')
np.linalg.det(A)

B = np.matrix('0.001 1;1 1')
print B

np.linalg.det(B)

[[ 0.001  1.   ]
 [ 1.     1.   ]]


-0.999

To illustrate the ill-conditioning of $A$ in another way let's consider two very close right-hand side vectors, $\vec{b}$:

$$
\begin{matrix}
x_1 +& x_2& = 2 &and& x_1 +& x_2 &=& 2\\
x_1 +&1.0001x_2& = 2 &and& x_1 +& 1.0001x_2 &=& 2.0001
\end{matrix}
$$

The system of equations on the left has the solution $x_1 = 2, x_2 = 0$, and the system of equations on the right has the solution $x_1 = x_2 = 1$. Notice that a change in the fifth digit of $\vec{b}$ was amplified to a change in the first digit of the solution. Even the most robust numerical method will have trouble with this sensitivity to small perturbations.

Now I would like to illustrate that even a well-conditioned matrix like $B$ can be ruined by a poor algorithm. Consider the matrix $B$ with the right-hand side vector, $\vec{b} = [1,2]$ and let's proceed through Gaussian elimination. We will start by multiplying $-10000$ times the first row and adding it to the second.

$$
\begin{pmatrix} 0.0001 & 1 & 1\\1&1&2\end{pmatrix} \rightarrow
\begin{pmatrix} 0.0001&1&1\\0&-9999&-9998\end{pmatrix} \rightarrow
_{-\frac{1}{9999}}
\begin{pmatrix} 0.0001&1&1\\0&-9999&-9998 \end{pmatrix} \rightarrow
\begin{pmatrix} 0.0001&1&1\\0&1&0.9999\end{pmatrix}
$$

by inspection we see that $x_2 = 0.9999$, now we can back substitute into the first equation

$x_1+10000x_2 = 10000$

and solve for $x_1$. The final solution is $x_1 = 1, x_2 = 0.9999$. Now let us do the same computation, this time on a computer that rounds the results of the computations to 3 decimal places (a machine epsilon of 0.0001), the resulting matrix after manipulations would be:

$$
\begin{pmatrix} 0.0001&1&1\\0&1&1\end{pmatrix}
$$

by inspection we see that $x_2 = 1$ and backsubstitution yields $x_1 = 0$. This is the incorrect solution brought on by the small pivot 0.0001. For this matrix Gaussian elimination is a poor algorithm. Fortunately there is an easy solution: exhange rows.

Now lets do the same computation on teh same computer that rounds off the computation to 3 decimal places. This time we will exchange the rows before proceeding with the Gaussian elimination. Eliminating some detail we can see:

$$
\begin{pmatrix}1&1&2\\0.0001&1&1\end{pmatrix}\rightarrow
\begin{pmatrix}1&1&2\\0&0.999&0.999\end{pmatrix}\rightarrow
\begin{pmatrix}1&1&2\\0&1&1\end{pmatrix}
$$

by inspection we see that $x_2 = 1$, and back substitution yields $x_1 = 1$. We can see that this solution differs from the exact solution only by the roundoff error of the computer. Exchanging rows to obtain the largest possible pivot is called *partial pivoting*. Exchanging both rows and columns to obtain the largest possible pivot is called *full pivoting*. Full pivoting will result in the most stable algorithm but requires more computations and the requirement of tracking column permutations. For most matrices partial pivoting is sufficient enough for stability.

## Gauss-Jordan Elimination

Gauss-Jordan elimination simply adds steps to the simple Gauss elimination procedure to produce a matrix that is in *reduced row echelon form*. This is done by eliminating values both above and below the pivots and ensuring that each pivot has the value 1. Starting where we ended on the exact solution of matrix $B$ earlier we can simply add two steps to produce a reduced row echelon matrix. First we subtract the second row from the first:

$$
\begin{pmatrix}0.0001&1&1\\0&1&0.9999\end{pmatrix}\rightarrow\begin{pmatrix}0.0001&0&0.0001\\0&1&0.9999\end{pmatrix}
$$

then we normalize the first equation to produce a 1 on the pivot:

$$
^{\frac{1}{0.0001}}
\begin{pmatrix}0.0001&0&0.0001\\0&1&0.9999\end{pmatrix}\rightarrow
\begin{pmatrix}1&0&1\\0&1&0.9999\end{pmatrix}
$$

now by inspection (and without the use of back substitution) we can see that the solutions are $x_1 = 1$ and $x_2 = 0.9999$ just as before.

For a linear system with $n$ equations and $m$ unknowns. The innermost loops (one subtraction and one multiplication) of a Gauss-Jordan elimination routine are executed $n^3$ and $n^2$ $m$ times. The corresponding loops in a Gaussian scheme are executed $\frac{1}{3}n^3$ and $\frac{1}{2}n^2m$ times, followed by a back substitution for a similar loop (one subtraction and one multiplication) taht occurs $\frac{1}{2}n^2$ times. If there are far more equations that unknowns the Gaussian scheme with back substitution enjoys roughly a factor of 3 advantage over the Gauss-Jordan, for $n=m$ Gaussian elimination with back substitution enjoys a factor of 6 advantage over Gauss-Jordan. For matrix inversion (discussed later) the two methods have identical efficiencies.

## Pseudocode for Gauss Elimination with Back Substitution and Partial Pivoting

Consider a linear system with $n$ equations and $n$ unknowns:
$$
E_1: a_{11}x_1+a_{12}x_2+...+a_{1n}x_n = a_{1,n+1}\\
E_2: a_{21}x_1+a_{22}x_2+...+a_{2n}x_n = a_{2,n_1}\\
...\\
E_n: a_{n1}x_1+a_{n2}x_2+...+a_{nn}x_n = a_{n,n+1}
$$

|Steps|   |   | |
|-----|---|---|-|
|1    |For $i = 1$, ..., $n$ do Steps 2-4|loop over columns in progression|
|     | 2 | Find $p$, where $p$ is the largest number with $i\leq p\leq n$ | search the current pivot and rows below the current one for the largest value|
|     | 3 | If $p\neq i$, then exchange row $i$ with row $p$ | now the largest value in the column is the new pivot|
|     | 4 | For $j=i+1, ..., n$ do Steps 5-6|the innermost loop, multiplication and subtraction|
|5    | | set $m_{ji} = a_{ji}/a_{ii}$| |
|6    | | Perform $E_j = ( E_j - m_{ji}E_i)$| |
|7    | | Set $x_n = a_{n,n+1}/a_{nn}$|start back substitution|
|8    | | For $i = n-1, ..., 1$ set $x_i=\frac{a_{i,n+1}-\sum\nolimits_{j=i+1}a_{ij}x_j}{a_{ii}}$| |

In [119]:
import numpy as np
def gauss_solve(A, b):
    
    #Concontanate the matrix A and right hand side column vector b into one matrix
    temp_mat = np.c_[A, b]
    
    (N, M) = temp_mat.shape
    
    #Loop over rows
    for i in range(N):
            
        #Find the pivot index by looking down the ith 
        #column from the ith row to find the maximum 
        #(in magnitude) entry.
        p = np.abs(temp_mat[i:, i]).argmax()
            
        #We have to reindex the pivot index to be the 
        #appropriate entry in the entire matrix, not 
        #just from the ith row down.
        p += i 
    
        #Swapping rows to make the maximal entry the 
        #pivot (if needed).
        if p != i:
            temp_row = temp_mat[p].copy()
            temp_mat[p] = temp_mat[i]
            temp_mat[i] = temp_row
            
        #Eliminate all entries below the pivot
        factor = temp_mat[i+1:, i] / temp_mat[i, i]
        temp_mat[i+1:] -= factor[:, np.newaxis] * temp_mat[i]
                
    #Allocating space for the solution vector
    x = np.zeros_like(b, dtype=np.double);

    #Here we perform the back-substitution.
    x[-1] = temp_mat[-1,-1] / temp_mat[-1, -2]
    
    #Looping over rows in reverse (from the bottom up), starting with the second to
    #last row, because the last row solve was completed in the last step.
    #for i in range(N-2, -1, -1):
    #    x[i] = (temp_mat[i,-1] - np.dot(temp_mat[i,i:-1], x[i:])) / temp_mat[i,i]
    x[-2::-1] = (temp_mat[-2::-1,-1] - np.dot(temp_mat[-2::-1,i:-1], x[i:])) / temp_mat[i,i]
        
    return x

In [131]:
x = np.zeros(10)
x[0] = 1
x[1] = 2
x[2:] = x[1:-1] - x[0:-2]
x

array([ 1.,  2.,  1., -2.,  0.,  0.,  0.,  0.,  0.,  0.])

In [121]:
xx = gauss_solve(x, np.array([0, 1, 2]))
yy = np.linalg.solve(x, np.array([0, 1, 2]))
print(xx)
print(yy)

[-283.59957158  106.95138115   15.03260587]
[-283.59957158  106.95138115   15.03260587]


## Pseudocode for Gauss-Jordan Elimination with Partial Pivoting

Consdier a linear system with $n$ equations and $n$ unknowns:

$$
E_1: a_{11}x_1+a_{12}x_2+...+a_{1n}x_n = a_{1,n+1}\\
E_2: a_{21}x_1+a_{22}x_2+...+a_{2n}x_n = a_{2,n_1}\\
...\\
E_n: a_{n1}x_1+a_{n2}x_2+...+a_{nn}x_n = a_{n,n+1}
$$

|Steps| | |
|-----|-|-|
|1|For $i=1,...,n$ do Steps 2-4|loop over columns in progression|
|2|Find $p$, where $p$ is the largest number with $i\leq p \leq n$|search the current pivot and the rows below the current one for the largest value|
|3|If $p\neq i$, then exchange row $i$ with row $p$|now the largest value in the column is the new pivot|
|4|For $j=1,...,n$ do Steps 5-6|the innermost loop, multiplication and subtraction|
|5|Perform $E_i = \frac{1}{a_{ii}E_i}$| |
|6|If $i \neq j$ perform $E_j-a_{ji}E_i$| |
|7|For $i = 1,...,n$ set $x_i = a_{i,n+1}$| |

## Matrix Inversion with Gauss-Jordan scheme

For matrix inversion, both the Gauss elimination with back-substitution and Gauss-Jordan schemes described previously have identical efficiencies. For this reason we will, for simplicities sake, only consider the process for matrix inversion using the Gauss-Jordan scheme.

All we have to do is augment an $A$ matrix with an identity matrix of the same dimensions and proceed with Gauss-Jordan elimination exactly as we have done previously. Consider the following $A$ matrix:

$A = \begin{pmatrix}1&3&2\\2&4&5\\10&7&3\end{pmatrix}$ we augment it with a 3x3 identity matrix $A|I$ and proceed with Gauss-Jordan Elimination.

$$
\begin{pmatrix}
1&3&2&1&0&0\\2&4&5&0&1&0\\10&7&3&0&0&1
\end{pmatrix} \rightarrow
\begin{pmatrix}
1&3&2&1&0&0\\0&-2&1&-2&1&0\\0&-23&-17&-10&0&1
\end{pmatrix}\rightarrow
$$

$$
\begin{pmatrix}
1&0&\frac{7}{2}&-2&\frac{3}{2}&0\\0&1&-\frac{1}{2}&1&-\frac{1}{2}&0\\0&0&-\frac{57}{2}&13&-\frac{23}{2}&1
\end{pmatrix}\rightarrow
\begin{pmatrix}
1&0&\frac{7}{2}&-2&\frac{3}{2}&0\\0&1&-\frac{1}{2}&1&-\frac{1}{2}&0\\0&0&-\frac{57}{2}&13&-\frac{23}{2}&1
\end{pmatrix} \rightarrow
\begin{pmatrix}
1&0&0&-\frac{23}{57}&\frac{5}{57}&\frac{7}{57}\\0&1&0&\frac{44}{57}&-\frac{17}{57}&-\frac{1}{57}\\0&0&1&-\frac{26}{57}&-\frac{23}{57}&-\frac{2}{57}
\end{pmatrix}
$$

Here I am neglecting the partial-pivoting which you would want to include to ensure stability, but we can see by inspection the right side 3x3 submatrix is the inverse of $A$. Verifying using the inverse routine build into ####*Python*#### we have:

In [1]:
A = [ [1,3,2],[2,4,5],[10,7,3] ] #creates matrix
b = A**-1

TypeError: unsupported operand type(s) for ** or pow(): 'list' and 'int'

In [20]:
from numpy import matrix
from numpy import linalg
A = matrix( [[1,3,2],[11,12,13],[21,22,23]] ) #creates matrix
print A.I                                     #Inverse of A

[[-0.33333333 -0.83333333  0.5       ]
 [ 0.66666667 -0.63333333  0.3       ]
 [-0.33333333  1.36666667 -0.7       ]]
